In [ ]:
import os
import json 
import argparse
import numpy as np
import math
import pickle
import datetime
import time
import logging
import itertools

You can specify an URL to an image

In [ ]:
url = 'https://cdn.topofart.com/images/artists/John_Singer_Sargent/paintings/sargent029.jpg'
import urllib.request
urllib.request.urlretrieve(url, "input_img.jpg")
imgpath = os.path.abspath("input_img.jpg")

Or specify an image from the styletransfer art dataset:

In [ ]:
imgpath = '/home/althausc/nfs/data/coco_17_medium/val2017_styletransfer/000000000785_050351.jpg'

Show the loaded image:

In [ ]:
from IPython.display import Image, display
display(Image(filename=imgpath))

Run keypoint & scene graph inference:

In [ ]:
import branchkpts.predict_search_modules as kptm
import branchgraphs.predict_search_modules as graphm

#Keypoint inference
kpt_annpath = kptm.predict(imgpath)
display(Image(filename="input_img_overlay.jpg"))

#Graph inference
g_annpath = graphm.predict_scenegraph(imgpath)

filterlabels = True #False
imgpath, anns = graphm.visualize_scenegraph(annpath, filterlabels)
display(Image(filename=imgpath))

with open(anns, 'r') as f:
    print(f.read())

---
Transforms the keypoint predictions to a geometric feature descriptor. <br>
Please specify following parameters:

    Method for constructing the feature vector
    PCA params, if necessary because of too many feature entries [Model only used when pca enabled]

Transforms the scene graph predictions into necessary Graph2Vec format. 
Furthermore only the best k predictions will be considered. <br>
Please specify following parameters:
    
    Relasnodes for transforming the predicates to nodes.
    

In [ ]:
#Transform poses to GPD descriptor
methodgpd = 0 #['JcJLdLLa_reduced', 'JLd_all']
pca_on = False #True
pca_model = '/home/althausc/master_thesis_impl/posedescriptors/out/08/27_13-49-24/modelpca64.pkl' 

gpdfile = kptm.transform_to_gpd(kpt_annpath, methodgpd, pca_on, pca_model)

#Transform graph predictions to G2V Format
relsasnodes = True
graphfile = graphm.transform_into_g2vformat(g_annpath,relsasnodes)

Search for images in the database with similar arrangements of persons. <br>
A database (Elastic Search) is queried with the given keypoint feature vector. <br>
Please specify following parameters: <br>
    1. tresh Threshold for minimum cos similarity scores (range=[0,1], only used by method 'COSSIM')
    2. method_search 
        - COSSIM uses cosinus similarity between query and each db image (cluster retrieval supported)
        - DISTSUM accumulates all L2-Distances from query to all db images


Search best matching image in reference to the computed scene graph. <br>
A pretrained model is used in query mode to infer best matching results.<br>
Please specify following parameters:<br>
    
    1. k Only obtain best k images according to pretrained G2V model(uses cosinus similarity)
    2. rw Reweight mode enabled, then retrieved images get reordered by similarities 
          in box & relationship label distribution
    3. rm Similarity metric between label distributions


In [ ]:
#Search with pose descriptors
method_search = 'COSSIM' #['COSSIM', 'DISTSUM']
tresh = 0.95
topkkpt_file = branchkpts.search(gpdfile, method_search, methodgpd, tresh)

#Search with graph embeddings
k = 10
rw = False #True
rm = 'jaccard' #'euclid' 
topngraph_file = search_topk(graphfile, k, reweight=rw, r_mode=rm)


Merge resultlists & Show images in ranking.

In [ ]:
import ipyplot
import branchtogether as bb
#results = '/home/althausc/master_thesis_impl/retrieval/out/09/03_16-45-46/result-ranking.json'

topk = 20
w_branches = 0.5 #0-> kpts only, 1-> graphs only
im_dir, mergedres = bb.merge_retrievalresults(topkkpt_file, topngraph_file, topk=topk, weight_branches = w_branches)

imgs, scores = bb.getImgs(im_dir, mergedres)


queryim = bb.drawborder(imgpath)
imgs.insert(0,queryim)


ipyplot.plot_images(imgs, labels=None, scores=scores, max_images=30, img_width=250)